# Strategy Generation Workshop

Generate thousands of trading strategies and find the best performers.

## What This Notebook Does:
1. Load historical market data
2. Generate 1000+ strategy combinations
3. Test all strategies using vectorized backtesting
4. Filter and rank by performance metrics
5. Export top strategies for further optimization

In [ ]:
# Setup
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from strategy_factory.generator import StrategyGenerator

# Display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Imports successful")

## 1. Load Market Data

In [ ]:
# Load historical data
df = pd.read_csv('../data/crypto/BTCUSD_5m.csv')

# Normalize column names
df.columns = df.columns.str.lower()

# Convert timestamp
if 'date' in df.columns:
    df = df.rename(columns={'date': 'timestamp'})

df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp')

print(f"📊 Loaded {len(df):,} bars")
print(f"   Period: {df.index[0]} to {df.index[-1]}")
print(f"   Columns: {', '.join(df.columns)}")

df.head()

## 2. Generate SMA Strategies

In [ ]:
# Initialize generator
generator = StrategyGenerator(initial_capital=10000, commission=0.001)

# Generate SMA crossover strategies
sma_results = generator.generate_sma_strategies(
    df=df,
    fast_range=[5, 10, 15, 20, 25],
    slow_range=[50, 100, 150, 200],
    verbose=True
)

# Display top 20
sma_results.head(20)

## 3. Generate RSI Strategies

In [ ]:
# Generate RSI mean reversion strategies
rsi_results = generator.generate_rsi_strategies(
    df=df,
    period_range=[7, 14, 21, 28],
    oversold_range=[20, 25, 30, 35],
    overbought_range=[65, 70, 75, 80],
    verbose=True
)

rsi_results.head(20)

## 4. Generate Breakout Strategies

In [ ]:
# Generate breakout strategies
breakout_results = generator.generate_breakout_strategies(
    df=df,
    lookback_range=[10, 20, 30, 50],
    breakout_pct_range=[0.5, 1.0, 1.5, 2.0],
    verbose=True
)

breakout_results.head(20)

## 5. Combine and Filter All Strategies

In [ ]:
# Combine all results
all_results = pd.concat([sma_results, rsi_results, breakout_results], ignore_index=True)
all_results = all_results.sort_values('sharpe_ratio', ascending=False)

print(f"📊 Total strategies generated: {len(all_results):,}")
print(f"\n🏆 Top performing strategy:")
print(f"   Type: {all_results.iloc[0]['params']}")
print(f"   Sharpe: {all_results.iloc[0]['sharpe_ratio']:.2f}")
print(f"   Return: {all_results.iloc[0]['total_return']:.2f}%")
print(f"   Max DD: {all_results.iloc[0]['max_drawdown']:.2f}%")

all_results.head(30)

In [ ]:
# Filter by quality criteria
filtered = generator.filter_strategies(
    results=all_results,
    min_sharpe=1.0,
    max_drawdown=20.0,
    min_trades=10,
    min_win_rate=0.45
)

filtered.head(20)

## 6. Export Results

In [ ]:
# Save top strategies
all_results.head(50).to_csv('../results/top_50_strategies.csv', index=False)
filtered.to_csv('../results/filtered_strategies.csv', index=False)

print("✅ Results exported:")
print("   - results/top_50_strategies.csv")
print("   - results/filtered_strategies.csv")

## Next Steps

1. **Optimize top strategies** - Use `02_strategy_optimization.ipynb` to optimize parameters
2. **Analyze performance** - Use `03_performance_analysis.ipynb` for detailed metrics
3. **Deploy to brokers** - Use `04_multi_broker_deployment.ipynb` to go live